# Accuracy Metrics Lab

## Measuring Classifier Performance Metrics

## Objective:

* Create and interpret a Confusion Matrix.
* Calculate Precision, and identify use cases for this metric.
* Calculate Recall, and identify use cases for this metric.
* Calculate Accuracy, and identify use cases for this metric.
* Calculate F-score, and identify use cases for this metric.

## Measuring Classifier Performance